In [6]:
import dicom
import os
import pandas as pd
import numpy as np

data_dir = 'C:\\Users\\wu1114\\Documents\\Data_Science_Bowl\\train\\train\\8\\study\\sax_54\\'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('C:\\Users\\wu1114\\Documents\\Data_Science_Bowl\\train.csv\\train.csv')
labels_df.head()

,Id,Systole,Diastole
0,1,108.3,246.7
1,2,54.6,137.2
2,3,32.7,99.3
3,4,57.7,154.5
4,5,83.3,235.5
